In [12]:
# import dependencies
import requests
import json
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# connect to API key
from config import gkey


In [13]:
# read zip codes csv
zipcodes_data = "ny_state_zipcodes.csv" 
zipcodes_df = pd.read_csv(zipcodes_data)
zipcodes_df.head()


,Zip Code
0,10001
1,10002
2,10003
3,10004
4,10005


In [14]:
# Add columns for income, # single, # no cards, # chinese
zipcodes_df["Avg Income ($)"] = ""
zipcodes_df["# Single"] = ""
zipcodes_df["# No Cars"] = ""
zipcodes_df["# Chinese"] = ""
zipcodes_df.head()


,Zip Code,Avg Income ($),# Single,# No Cars,# Chinese
0,10001,,,,
1,10002,,,,
2,10003,,,,
3,10004,,,,
4,10005,,,,


In [ ]:
# # add zero to first three zips to create 5 digit zips 
# zipcodes_df['Zip Code']=zipcodes_df['Zip Code'].apply(lambda x: '{0:0>5}'.format(x))
# zipcodes_df.head()


In [15]:
zipcodes_df = zipcodes_df.astype({"Zip Code": object})
zipcodes_df.dtypes

Zip Code          object
Avg Income ($)    object
# Single          object
# No Cars         object
# Chinese         object
dtype: object

In [16]:
# establish base URL 
base_url = "https://api.census.gov/data/2017/acs/acs5/profile"

# print the response object to the console
response = requests.get(base_url)
print(response)


<Response [200]>


In [17]:
# loop through zip codes and pull other variables for each 

base_url = "https://api.census.gov/data/2017/acs/acs5/profile"

avg_income = "DP03_0063E"   
single_hh = "DP02_0011E"
hh_no_vehicles = "DP04_0058E"
chinese_pop = "DP05_0046E"

for index, row in zipcodes_df.iterrows():    
    
    zip = row['Zip Code']
    
    query_url = f"{base_url}?get={avg_income},{single_hh},{hh_no_vehicles},{chinese_pop},NAME&for=zip%20code%20tabulation%20area:{zip}&key={gkey}"
    #query_url = f"{base_url}?get={avg_income},NAME&for=zip%20code%20tabulation%20area:{zip}&key={gkey}"
    
    response = requests.get(query_url).json()
    #pprint(response)
    
    try:
        print(f"Finding demo data for zip code: {zip}")
        
        zipcodes_df.loc[index, "Avg Income ($)"] = response[1][0]
        zipcodes_df.loc[index, "# Single"] = response[1][1]
        zipcodes_df.loc[index, "# No Cars"] = response[1][2]
        zipcodes_df.loc[index, "# Chinese"] = response[1][3]

        
    except ("JSONDecodeError"):
        print("Problem with data at this zip... skipping.")
        
    print("-------------------------------------------")
    

Finding demo data for zip code: 10001
-------------------------------------------
Finding demo data for zip code: 10002
-------------------------------------------
Finding demo data for zip code: 10003
-------------------------------------------
Finding demo data for zip code: 10004
-------------------------------------------
Finding demo data for zip code: 10005
-------------------------------------------
Finding demo data for zip code: 10006
-------------------------------------------
Finding demo data for zip code: 10007
-------------------------------------------


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [18]:
# output data frame with info added in new columns
zipcodes_df


,Zip Code,Avg Income ($),# Single,# No Cars,# Chinese
0,10001,161227,6554,10193,2467
1,10002,65718,14219,27306,30774
2,10003,182531,15067,20895,3166
3,10004,191760,808,1287,381
4,10005,201734,1728,3538,699
5,10006,196771,881,1527,208
6,10007,376573,1029,1961,446
7,10008,,,,
8,10009,,,,
9,10010,,,,
